In [1]:
import pandas as pd

df = pd.read_csv('Oregon Hwy 26 Crash Data for 2019.csv')

# print(df.head(30))
# df.dropna(axis=1,how='all')

In [2]:
df.duplicated().sum()

0

In [3]:
CrashesDF = df[df['Record Type'] == 1]
VehiclesDF = df[df['Record Type'] == 2]
ParticipantsDF = df[df['Record Type'] == 3]

In [4]:
CrashesDF = CrashesDF.dropna(axis=1,how='all')
VehiclesDF = VehiclesDF.dropna(axis=1,how='all')
ParticipantsDF = ParticipantsDF.dropna(axis=1,how='all')

In [5]:
# # print(CrashesDF)
# print(VehiclesDF)
# print(ParticipantsDF)

# Existence assertions:

In [6]:
#1. Every crash occurred on a date

CrashesDF[['Crash Month','Crash Day','Crash Year']].isnull().values.any()


False

In [7]:
#2. Every crash has its record id (#serial no) 

CrashesDF['Serial #'].isnull().values.any()

False

In [8]:
#3. Every crash has its vehicle ID

VehiclesDF['Vehicle ID'].isnull().values.any()

False

In [9]:
#4. Every crash has its participant ID 

ParticipantsDF['Participant ID'].isnull().values.any()

False

In [10]:
#5. Every crash has longitude and latitude value

CrashesDF[['Longitude Degrees','Longitude Minutes', 
           'Longitude Seconds', 'Latitude Degrees', 
           'Latitude Minutes', 'Latitude Seconds']].isnull().values.any()

False

# Limit assertions:

In [11]:
#1. Every crash occurred during year 2019

year = 2019
CrashesDF['Crash Year'].eq(year).all()

True

In [12]:
#2. Every crash occured on Highway 26

highway_no = 26
CrashesDF['Highway Number'].eq(highway_no).all()

True

In [13]:
#3. Every crash participant’s age range between 0 -125

min_age = ParticipantsDF['Age'].min()
max_age = ParticipantsDF['Age'].max()
print(min_age, max_age)

ParticipantsDF['Age'].unique()
# ParticipantsDF['Age'].describe()

0.0 9.0


array([ 6.,  2., nan,  9.,  0.,  4.,  1.])

# Intra-record assertions:

In [14]:
#1. If a crash record has a latitude coordinate then it should also have a longitude coordinate

CrashesDF[['Longitude Degrees','Longitude Minutes', 
           'Longitude Seconds', 'Latitude Degrees', 
           'Latitude Minutes', 'Latitude Seconds']].notnull().values.any()

True

In [15]:
#2. Every crash has a unique ID

CrashesDF['Crash ID'].unique().all()

True

In [16]:
#3. Every crash id has a serial no

CrashesDF[["Crash ID","Serial #"]].isnull().values.any()

False

In [17]:
# For Longitude coordinates, minutes and seconds fields 
# are also populated if the degrees field is populated.

CrashesDF[['Longitude Degrees', 
           'Longitude Minutes', 
           'Longitude Seconds']].notnull().values.any()

True

# Inter-record check assertions:

In [18]:
#1. Every vehicle listed in the crash data was part of a known crash

VehiclesDF[['Vehicle ID', 'Crash ID']].isnull().values.all()

False

In [19]:
#2. Each crash ID has at least one vehicle associated with it

VehiclesDF[['Crash ID', 'Vehicle ID']].groupby(['Crash ID']).count().min().eq(1)

Vehicle ID    True
dtype: bool

In [20]:
#3. Each crash ID has at least one participant 

ParticipantsDF[['Crash ID', 'Participant ID']].groupby(['Crash ID']).count().min().eq(1)

Participant ID    True
dtype: bool

In [21]:
#4. Participant ID may or may not be associated with a Vehicle

ParticipantsDF[['Vehicle ID', 'Participant ID']].eq(0).values.any()

True

# Summary Assertions:

In [22]:
#1. There were thousands of crashes but not millions

CrashesDF['Crash ID'].count().max()

508

In [23]:
#2. Two or more vehicles are involved in a crash on average

# VehiclesDF.groupby(['Crash ID']).count()

VehiclesDF_Subset = VehiclesDF[['Crash ID','Vehicle ID']]
VehiclesDF_SubsetGroupBy = VehiclesDF_Subset.groupby(['Crash ID']).count()
VehiclesDF_SubsetGroupBy.mean().ge(2)

Vehicle ID    False
dtype: bool

In [24]:
#3. The number of crashes tends to be higher in weekdays than weekend 

CrashesDF[['Crash ID', 'Week Day Code']].groupby(['Week Day Code']).count()


,Crash ID
Week Day Code,
1.0,60
2.0,71
3.0,75
4.0,74
5.0,68
6.0,77
7.0,83


# Statistical distribution assertions:

In [25]:
#1. crashes are evenly/uniformly distributed throughout the months of the year

CrashesDF[['Crash ID','Crash Month']].groupby(['Crash Month']).count()

,Crash ID
Crash Month,
1.0,56
2.0,46
3.0,57
4.0,53
5.0,41
6.0,36
7.0,37
8.0,33
9.0,34


In [26]:
#2. Crashes are uniform throughout the day

CrashesDF[['Crash ID','Crash Hour']].groupby(['Crash Hour']).count()

,Crash ID
Crash Hour,
0.0,11
1.0,5
2.0,5
3.0,4
4.0,2
5.0,9
6.0,8
7.0,19
8.0,26


In [27]:
#3. The number of crashes tends to be higher in weekdays than weekend 

CrashesDF[['Crash ID', 'Week Day Code']].groupby(['Week Day Code']).count()

,Crash ID
Week Day Code,
1.0,60
2.0,71
3.0,75
4.0,74
5.0,68
6.0,77
7.0,83


# Output of the validated data to new files

In [28]:
CrashesDF.to_csv('CrashesData.csv')

In [29]:
VehiclesDF.to_csv('VehiclesData.csv')

In [30]:
ParticipantsDF.to_csv('Participants.csv')